# Lab: Solving the Multi-Dimensional Knapsack Problem with QAOA

Welcome! In this lab, you'll tackle a challenging optimization problem that's a step up from the classic knapsack problem: the **Multi-Dimensional Knapsack Problem (MDKP)**. You'll learn how to formulate this problem mathematically, model it using Qiskit's optimization module, and finally, solve it using a quantum algorithm, QAOA.

### Learning Goals 🎯

* Understand the mathematical formulation of MDKP.
* Learn how to translate a constrained optimization problem into a `QuadraticProgram`.
* Use Qiskit to automatically convert the problem into a QUBO (Hamiltonian).
* Apply the `MinimumEigenOptimizer` with QAOA to find a solution.
* Interpret the results from a quantum optimization routine.

# 🚀 1. The Multi-Dimensional Knapsack Problem (MDKP): Real-World Resource Optimization

## 🌍 Why This Matters

You’re not just solving an abstract puzzle — you’re tackling a problem that logistics managers, disaster relief coordinators, aerospace engineers, and even AI startups face every day:

> **“How do I get the most value out of limited, multi-faceted resources?”**

Whether you’re packing a Mars rover, allocating a startup’s budget across engineering and marketing, or distributing medical supplies after a hurricane — you’re constantly juggling **multiple scarce resources** against **competing priorities**.

That’s the **Multi-Dimensional Knapsack Problem (MDKP)**.

---

## 🧳 The Core Challenge

You’re given a set of **items** — each with:

- A **value** (impact, profit, utility)
- A **cost in multiple dimensions** (weight, volume, power, time, budget, carbon footprint, etc.)

You have **limited capacity in each dimension**.

Your goal: **Choose which items to take to maximize total value — without breaking any of your limits.**

This isn’t science fiction. This is supply chain optimization. This is mission planning. This is smart resource allocation in a constrained world.

---

## 📐 Mathematical Formulation

Let’s formalize it.

- You have **`n` items**, indexed by `i = 1, ..., n`.
- Each item `i` has a **value** `vᵢ` — what you gain by selecting it.
- There are **`m` resource constraints**, indexed by `j = 1, ..., m` — e.g., weight, volume, power, budget.
- Each item `i` consumes `wᵢⱼ` units of resource `j`.
- Each resource `j` has a maximum capacity `Cⱼ`.

You choose a binary variable for each item:  
> `xᵢ = 1` → you take the item  
> `xᵢ = 0` → you leave it behind

---

### 🎯 Objective: Maximize Total Value

$$
\max \sum_{i=1}^{n} v_i x_i
$$

---

### ⚖️ Subject to: Respect All Resource Limits

For each resource `j` (e.g., kg, m³, kW, $):

$$
\sum_{i=1}^{n} w_{ij} x_i \leq C_j
$$

---

### 📦 Real-World Interpretations

| Scenario             | Items                     | Value               | Constraint 1       | Constraint 2       | Constraint 3       |
|----------------------|---------------------------|---------------------|--------------------|--------------------|--------------------|
| **Space Mission**    | Scientific instruments    | Science return      | Mass (kg)          | Volume (m³)        | Power (kW)         |
| **Disaster Relief**  | Medical kits, tents, food | Lives saved         | Truck weight       | Pallet space       | Refrigeration kW   |
| **Tech Startup**     | Features to build         | User engagement     | Engineering hours  | Cloud cost ($)     | Data bandwidth     |
| **E-commerce**       | Products to promote       | Expected revenue    | Ad budget ($)      | Banner space (px)  | Compliance score   |

---

## 🧪 Example: Humanitarian Aid Delivery

> 💡 *Let’s ground this in a scenario students can feel: delivering aid after a natural disaster.*

You’re coordinating a relief flight into a flood zone. You have 4 critical supplies to choose from. Your cargo plane has strict limits on **weight** and **volume**.

Here’s your inventory:

| Item                | Value (Lives Saved) | Weight (kg) | Volume (m³) |
|---------------------|---------------------|-------------|-------------|
| Water Purifiers     | 7                   | 3           | 1           |
| Emergency Tents     | 5                   | 2           | 3           |
| Antibiotic Kits     | 6                   | 4           | 2           |
| Solar Lanterns      | 3                   | 1           | 4           |

**Constraints**:
- Max Weight: **6 kg**
- Max Volume: **7 m³**

> 🤔 **Question**: Which combination saves the most lives without overloading the plane?

Let’s compute a few options:

- **Water Purifiers + Tents**: Value = 12, Weight = 5, Vol = 4 → ✅ Feasible  
- **Antibiotic Kits + Lanterns**: Value = 9, Weight = 5, Vol = 6 → ✅ Feasible  
- **Water Purifiers + Antibiotics**: Value = 13, Weight = 7 ❌ → Overweight!

✅ **Optimal Solution**: Water Purifiers + Tents → **Value = 12**

*(Note: Antibiotics alone = Value 6, Weight 4, Vol 2 — also feasible but suboptimal)*

---

## 🤖 Why Solve This with Quantum?

Classical solvers (like dynamic programming or integer programming) work well for small instances — but real-world logistics problems involve **hundreds of items and dozens of constraints**. The solution space explodes exponentially.

Quantum algorithms like **QAOA** and **VQE** offer a new way to explore this space — not by brute force, but by leveraging quantum superposition and interference to *bias the search toward high-value, feasible solutions*.

In this tutorial, you’ll encode this humanitarian logistics problem into a **quantum Hamiltonian**, run **QAOA**, and see if the quantum algorithm can find the optimal — or near-optimal — relief package.




> 💡 **Spoiler**: If the quantum algorithm picks Water Purifiers + Tents — you’ve saved 12 lives. 🚁💙



--- 
## 2. Modeling the Problem with Qiskit Optimization

Manually converting this problem into a Hamiltonian would be complex. We'd need to introduce slack variables for the inequality constraints ($\le$) and then formulate a penalty term for each.

Thankfully, the `qiskit_optimization` module can handle this for us! Our first step is to describe the problem using the `QuadraticProgram` class.

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy as np

# Qiskit imports
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization import QuadraticProgram

In [3]:
# Problem instance definition
values = np.array([7, 5, 6, 3])
weights = np.array([
    [3, 2, 4, 1],  # Constraint 1 (e.g., weight in kg)
    [1, 3, 2, 4]   # Constraint 2 (e.g., volume in m^3)
])
capacities = np.array([6, 7])
n_items = len(values)
n_constraints = len(capacities)

# Create a Qiskit QuadraticProgram
mdkp_model = QuadraticProgram(name="MultiDimensionalKnapsack")

# Define the binary decision variables
# One variable for each item: x_i = 1 if item i is selected, 0 otherwise
mdkp_model.binary_var_list(n_items, name='x')

# Define the objective function to MAXIMIZE the total value
mdkp_model.maximize(linear=values)

# Add the constraints
for j in range(n_constraints):
    mdkp_model.linear_constraint(
        linear=weights[j],      # The weights for this constraint
        sense="<=",             # The inequality relation
        rhs=capacities[j],      # The capacity for this constraint
        name=f"constraint_{j}"
    )

# Let's print the model to see if it's correct
print(mdkp_model.prettyprint())

Problem name: MultiDimensionalKnapsack

Maximize
  7*x0 + 5*x1 + 6*x2 + 3*x3

Subject to
  Linear constraints (2)
    3*x0 + 2*x1 + 4*x2 + x3 <= 6  'constraint_0'
    x0 + 3*x1 + 2*x2 + 4*x3 <= 7  'constraint_1'

  Binary variables (4)
    x0 x1 x2 x3



--- 
## 3. Solving with a Quantum Algorithm: QAOA

Now that we have the problem modeled, we can use the `MinimumEigenOptimizer` to solve it. This high-level class orchestrates the entire process:

1.  It takes our `QuadraticProgram` with its constraints.
2.  It automatically converts it into a QUBO (an unconstrained problem with penalties), which is equivalent to an Ising Hamiltonian.
3.  It uses a specified quantum algorithm (we'll choose QAOA) to find the ground state of this Hamiltonian.
4.  It maps the result back to our original variables, giving us the solution.

Let's set up the QAOA solver and the optimizer.

In [4]:
# We'll use the classic QAOA algorithm
# It requires a classical optimizer and a quantum backend (Sampler)
optimizer = COBYLA()
sampler = Sampler()
qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=2)

# Create the MinimumEigenOptimizer, which will use our QAOA instance to solve the problem
qaoa_solver = MinimumEigenOptimizer(min_eigen_solver=qaoa)

# Solve the problem!
result = qaoa_solver.solve(mdkp_model)

# Print the solution
print(f"Optimal solution: {result.x}")
print(f"Optimal value: {result.fval}")
print(result.prettyprint())

Optimal solution: [1. 1. 0. 0.]
Optimal value: 12.0
objective function value: 12.0
variable values: x0=1.0, x1=1.0, x2=0.0, x3=0.0
status: SUCCESS


The output will show you the optimal selection of items (`result.x`) and the maximum value achieved (`result.fval`). A solution of `[1. 0. 1. 0.]` means we should pick the first and third items.

## 4. Your Turn! 🚀 — Load the Mars Cargo Module

You’re the mission planner for a resupply mission to the **Mars Base Alpha**.  
You have **5 critical items** you can load into the cargo module.  
Each item has a **scientific value** (how much it advances the mission) and consumes resources across **three key dimensions**:

- **Mass** (kg) — your lander can carry at most **12 kg**
- **Volume** (m³) — your storage bay holds at most **10 m³**
- **Power Draw** (kW) — your power budget is **11 kW**

> ⚠️ Exceed any limit, and the mission is scrubbed. Choose wisely.

---

### 🧳 Available Items

| Item # | Item Name          | Scientific Value | Mass (kg) | Volume (m³) | Power (kW) |
|--------|--------------------|------------------|-----------|-------------|------------|
| 0      | Soil Analyzer      | 12               | 5         | 2           | 4          |
| 1      | Drone Scout        | 10               | 3         | 4           | 5          |
| 2      | BioLab Kit         | 15               | 6         | 5           | 2          |
| 3      | Comm Booster       | 8                | 3         | 1           | 6          |
| 4      | Emergency Battery  | 5                | 2         | 3           | 1          |

---

### ✅ Your Task — Fill in the Blanks

In the code cell below (which you will complete), you must define:

#### 1. `values` → What goes here?
> This list holds the **scientific value** of each item, in order (item 0 to item 4).  
> Example: The Soil Analyzer (item 0) has value `12`, so `values[0] = 12`.

✅ **You write**: `values = [ ?, ?, ?, ?, ? ]`

---

#### 2. `weights` → What goes here?

> This is a **2D list (3 rows × 5 columns)**, where:
> - Each **row `j`** corresponds to **constraint `j`**, in this order: **[Mass, Volume, Power]**
> - Each **column `i`** corresponds to **item `i`**
>
> So: `weights[j][i]` = amount of **constraint `j`** consumed by **item `i`**

> For example:
> - `weights[0][0]` = Mass consumed by item 0 
> - `weights[1][0]` = Volume consumed by item 0 
> - `weights[2][0]` = Power consumed by item 0 

✅ **You write**:
```python
weights = [
    [? , ?, ?, ? , ?],  # constraint 0
    [? , ?, ?, ? , ?],  # constraint 1
    [? , ?, ?, ? , ?]   # constraint 2
]
```

#### 3. Constraint Sense → What goes here?

> In the code, you’ll add constraints like this:
> ```python
> model.linear_constraint(linear=weights[j], sense="?", rhs=capacities[j], ...)
> ```
>
> You must replace `"?"` with the correct **constraint sense**:
> - `"<="` — total consumption must be **at most** the capacity (upper bound)
> - `">="` — total consumption must be **at least** the capacity (lower bound)
> - `"=="` — total consumption must **exactly equal** the capacity





<details>
<summary>🔍 Click here for hint</summary>

> 🤔 **Think**:  
> Are you trying to **respect a limit** (e.g., “cannot exceed 12 kg”)?  
> → Then you need an **upper bound**: `<=`

> ✅ In this space mission:  
> - “Lander can carry *at most* 12 kg” → `total_mass <= 12`  
> - “Storage holds *no more than* 10 m³” → `total_volume <= 10`  
> - “Power draw must stay *within* 11 kW” → `total_power <= 11`

✅ **You write**: `sense = "<="`

> ⚠️ Choosing `>=` would force the solution to *exceed* the limit — which is physically impossible and breaks the model.

In [ ]:
# --- Exercise Cell ---

# 1. Define the problem variables for the new instance
new_values = np.array([ ?, ?, ?, ?, ? ])  # TODO: Add the new values array
new_weights = np.array([
    [ ?, ?, ?, ?, ? ],  # ← Constraint 0
    [ ?, ?, ?, ?, ? ],  # ← Constraint 1
    [ ?, ?, ?, ?, ? ]   # ← Constraint 2
])
new_capacities = np.array([ ?, ?, ? ])   # TODO: Add the new capacities array
n_items_new = len(new_values)

# 2. Create a new QuadraticProgram named "NewMDKP"
new_mdkp_model = QuadraticProgram(name="NewMDKP")

# 3. Add the binary variables to the model
# Hint: Use new_mdkp_model.binary_var_list(...)
new_mdkp_model.binary_var_list(n_items_new, name='x')

# 4. Set the objective function (remember to maximize)
# Hint: Use new_mdkp_model.maximize(...)
new_mdkp_model.maximize(linear=new_values)

# 5. Add the three linear constraints using a for loop
# TODO: What should the constraint sense be? '<=', '>=', or '=='?
#       Think: Are we setting a MAXIMUM limit or MINIMUM requirement?
for j in range(len(new_capacities)):
    new_mdkp_model.linear_constraint(
        linear=new_weights[j],
        sense="?",                         # TODO: Replace "?" with correct sense: '<=', '>=', or '=='
        rhs=new_capacities[j],
        name=f"constraint_{j}"
    )

# 6. The QAOA solver is already configured from the previous step.
#    Use the 'qaoa_solver' to solve your new model 'new_mdkp_model'.
new_result = qaoa_solver.solve(new_mdkp_model)

# 7. Print your results
print("--- New MDKP Solution ---")
print(f"Optimal selection: {new_result.x}")
print(f"Maximum value: {new_result.fval}\n")

# 8. (Bonus) Verify the solution!
#    Check if the total weight for each constraint is within its capacity.
selected_items_indices = np.where(new_result.x == 1)[0]
print("Verification:")
for j in range(len(new_capacities)):
    total_weight = np.sum(new_weights[j][selected_items_indices])
    is_valid = total_weight <= new_capacities[j]
    print(f"Constraint {j}: Total Weight = {total_weight}, Capacity = {new_capacities[j]}. Valid: {is_valid}")

**Good luck!** Compare your results with your peers. Did you find the best combination of items?

## Solution

<details>
<summary>🔍 Click to reveal solution</summary>

#### Solution

```python
--- Exercise Cell ---

# 1. Define the problem variables for the new instance
new_values = np.array([12, 10, 15, 8, 5]) # TODO: Add the new values array
new_weights = np.array([                   # TODO: Add the new weights matrix
    [5, 3, 6, 3, 2],
    [2, 4, 5, 1, 3],
    [4, 5, 2, 6, 1]
])
new_capacities = np.array([12, 10, 11]) # TODO: Add the new capacities array
n_items_new = len(new_values)

# 2. Create a new QuadraticProgram named "NewMDKP"
new_mdkp_model = QuadraticProgram(name="NewMDKP")

# 3. Add the binary variables to the model
# Hint: Use new_mdkp_model.binary_var_list(...)
new_mdkp_model.binary_var_list(n_items_new, name='x')

# 4. Set the objective function (remember to maximize)
# Hint: Use new_mdkp_model.maximize(...)
new_mdkp_model.maximize(linear=new_values)

# 5. Add the three linear constraints using a for loop
# Hint: Use new_mdkp_model.linear_constraint(...)
for j in range(len(new_capacities)):
    new_mdkp_model.linear_constraint(
        linear=new_weights[j],
        sense="<=",
        rhs=new_capacities[j],
        name=f"constraint_{j}"
    )

# 6. The QAOA solver is already configured from the previous step.
#    Use the 'qaoa_solver' to solve your new model 'new_mdkp_model'.
new_result = qaoa_solver.solve(new_mdkp_model)

# 7. Print your results
print("--- New MDKP Solution ---")
print(f"Optimal selection: {new_result.x}")
print(f"Maximum value: {new_result.fval}\n")

# 8. (Bonus) Verify the solution!
#    Check if the total weight for each constraint is within its capacity.
selected_items_indices = np.where(new_result.x == 1)[0]
print("Verification:")
for j in range(len(new_capacities)):
    total_weight = np.sum(new_weights[j][selected_items_indices])
    is_valid = total_weight <= new_capacities[j]
    print(f"Constraint {j}: Total Weight = {total_weight}, Capacity = {new_capacities[j]}. Valid: {is_valid}")
```

</details>